In [94]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
import statsmodels.api as sm
sns.set(style="ticks")
%matplotlib inline
import pickle
from sklearn import preprocessing

In [95]:
pm = pd.read_csv('finished_pm.csv')

In [96]:
pm.head()

,Parameter Name,Site_keys,ppm,forest,open_land,water,wetland,saltwater/sandybeach,recreational,marina,...,transportation,mining,waste,crop_land,cemetary,golfcourse,high_density_residential,medium_density_residential,low_density_residential,industrial
0,PM2.5 - Local Conditions,[ 42.452017 -73.255089],0.005385,0.000000,0.0,0.000000,0.0,0,0,0,...,0.0,0,0,0.0,0,0,0,0.170000,0,0.00
1,PM2.5 - Local Conditions,[ 42.448009 -73.254108],0.005602,0.000000,0.0,0.000000,0.0,0,0,0,...,0.0,0,0,0.0,0,0,0,0.370000,0,0.00
2,PM2.5 - Local Conditions,[ 41.685707 -71.169235],0.006388,0.000000,0.0,0.120000,0.0,0,0,0,...,0.0,0,0,0.0,0,0,0,0.640000,0,0.01
3,PM2.5 - Local Conditions,[ 42.474642 -70.970816],0.005536,0.000000,0.0,0.010000,0.0,0,0,0,...,0.0,0,0,0.0,0,0,0,0.910000,0,0.00
4,PM2.5 - Local Conditions,[ 42.770837 -71.10229 ],0.010355,0.016129,0.0,0.008065,0.0,0,0,0,...,0.0,0,0,0.0,0,0,0,0.782258,0,0.00


In [97]:
#function to rename columns that have backslashes in them
def rename_again(df):
    df=df.rename(columns = {'saltwater/sandybeach':'saltwater_sandybeach'})
    df=df.rename(columns = {'urban_public/institution':'urban_public_institution'})
    return df

In [98]:
pm = rename_again(pm)

In [99]:
# x = pm.iloc[:, 3:].astype(float)
# y = pm.iloc[:, 2]
# y = (y*100)
all_predictors = ['forest', 'open_land', 'water', 'wetland', 'saltwater_sandybeach', 'recreational', 'marina', 'transitional', 'urban_public_institution', 'commercial', 'transportation', 'mining', 'waste', 'crop_land', 'cemetary', 'golfcourse', 'high_density_residential', 'medium_density_residential', 'low_density_residential', 'industrial']
train, test = train_test_split(pm, test_size=0.30, random_state=0)
y_train = train['ppm'].values
y_test = test['ppm'].values


In [100]:

# x_train = np.delete(x_train, 0, 1)
# print ((x_train))



In [101]:
# DUMP TRAIN AND TEST VALUES AND SAVE THEM TO USE CONSISTENTLY
# pickle.dump(x_train, open("pic_train_x_params", 'wb', 2))
# pickle.dump(x_test, open("pic_test_x_params", 'wb', 2))
# pickle.dump(y_train, open("pic_train_y_params", 'wb', 2))
# pickle.dump(y_test, open("pic_test_y_params", 'wb', 2))

In [102]:
# x_train = pickle.load(open('pic_train_x_params', 'rb'))
# y_train = pickle.load(open('pic_train_y_params', 'rb'))


In [103]:
#function that creates an instance of a statmodels.api Ordinary Least Squares Model
def reg_m(y, x):
    model = sm.OLS(y, x).fit()
    #fits simple ordinary least squares model
    predictions = model.predict(x)
    #makes predictions for y based on x
    return(model.summary())


In [104]:
model = reg_m(y_train ,x_train)
print (model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     6.023
Date:                Thu, 13 Jul 2017   Prob (F-statistic):           4.25e-06
Time:                        10:44:50   Log-Likelihood:                 392.33
No. Observations:                 138   AIC:                            -770.7
Df Residuals:                     131   BIC:                            -750.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const               0          0        nan        n

/Users/taraippolito/anaconda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1471: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/taraippolito/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/taraippolito/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/taraippolito/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/taraippolito/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [105]:
'''Method that takes in a list of all the predictors left in the model, and the model, 
spits out the lowest p valued predictor and lowest p value'''
def find_low_p(all_preds, model_OLS):
    #initiates lowest p value
    lowestp = 1
    lowestlu = ' '
    #iterates through predictor variables
    for i in range(len(all_preds)):
        
        #compares pvalues between ith and ith +1 term
        if (model_OLS.pvalues[i] < lowestp):
            
            #reassigns lowest pvalue variable, gets index of lowest pvalue variable
            lowestp = model_OLS.pvalues[i]
            lowestlu = all_preds[i]
        else:
            continue
    return lowestp , lowestlu

In [106]:
# find_low_p(all_predictors, m)

# x_train = x_train[predictors].values #update parameters to no longer include the lowest p valued predictor


# print (x_train)

In [107]:
"""Input: x data set, y data set, list of predictor variables
Action: Runs a ridge regression through initial data set, computes pvalue, deletes predictor with least pvalue
returns: models - one for each new data set created after deleting
"""
def backwards_var_sel(x, y, all_predictors, train):
    models = [] #to hold all the models
    for i in range(len(all_predictors)-1):
        #fit a MLR to data set
        OLS_model = sm.OLS(y, x).fit()
        
        #save new model in a list
        models.append(OLS_model)
        
        #finds lowest pvalue and index of lowest pvalue
        lowestp, lowestlu = find_low_p(all_predictors, OLS_model)
        
        #removes variable with lowest p value
        index_td = all_predictors.index(lowestlu)
        del all_predictors[index_td]
        
        #updates list of x values, not including variable with lowest pvalue
        x = train[all_predictors].values
        
    return models

In [108]:
prelim_models = backwards_var_sel(x_train, y_train, predictors, train)
print (prelim_models)

/Users/taraippolito/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/taraippolito/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/taraippolito/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/taraippolito/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


IndexError: index 16 is out of bounds for axis 0 with size 16

## """Bayesian Linear Model"""
ridge_regression = Ridge(alpha=1.0, fit_intercept=True)
ridge_regression.fit(x_train, y_train)

In [98]:
print ('Ridge regression model:\n {} + {}^T . x'.format(ridge_regression.intercept_, ridge_regression.coef_))

Ridge regression model:
 1.580468914760643 + [-0.48959421 -0.57961581 -0.50722643 -0.01516038  0.          0.          0.
 -0.08696047  0.30857794 -0.76260961 -0.60247796  0.          0.         -0.0594071
  0.          0.          0.         -0.79518611  0.         -1.16632488]^T . x


In [99]:
print ('Train R^2: {}'.format(ridge_regression.score(x_train, y_train)))

Train R^2: 0.06579678012203594


In [100]:
#normalize and scale L2
normalized_x = preprocessing.normalize(x_train, norm = 'l2')
scaled_norm_x = preprocessing.scale(normalized_x)

In [101]:
print (scaled_norm_x[0])

[-0.52905261 -0.11921138 -0.15270835 -0.12097729  0.          0.          0.
 -0.12128807 -0.38207965 -0.51840855 -0.48826937  0.          0.
 -0.37666282  0.          0.          0.          1.37195492  0.
 -0.39554259]


In [102]:
"""Bayesian Linear Model"""
ridge_regression_scaled = Ridge(alpha=1.0, fit_intercept=True)
ridge_regression_scaled.fit(scaled_norm_x, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [103]:
print ('Train R^2: {}'.format(ridge_regression_scaled.score(scaled_norm_x, y_train)))

Train R^2: 0.08580808974377219
